In [ ]:
library(ggtree)
library(ape)
library(BiocManager)
library(dplyr)
library(stringr)
library(tidyverse)
library(IRdisplay)
library(readr)
library(caret)
library(randomForest)
library(tibble)
library(ggplot2)
library (pheatmap)
library(dplyr)
library(RColorBrewer)


In [ ]:
SL_E <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/SL_E.csv')
ALL_E <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/ALLE_II_F1.1.csv')
SL_NS1 <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/SL_DENV2_NS1.csv')
ALL_NS1 <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/ALLNS1_II_F1.1.csv')

In [ ]:
# Lolipop plot of All II_F1.1 E mutations and occurence frequency
#--- Defining E Gene Domains and Colors (using your provided info) ---
E_domains_df <- tibble(
  domain_name_E = c("Domain Ia", "Domain Ib", "Domain Ic", "Fusion loop a", "Domain IIa", "Domain IIb", "Fusion loop b", "Domain III", "Stem", "Transmembrane"),
  start = c( 1, 131, 271, 70,  52, 192, 242, 299, 395, 450), # Corrected Vector Length
  end   = c(52, 192, 299, 82, 131, 271, 248, 394, 449, 495)  # Corrected Vector Length

)

domain_colors_E <- c(
  "Domain Ia" = "#e4fa05", "Domain Ib" = "#fa8e06", "Domain Ic" = "#f4ee51",
  "Fusion loop a" ="#8ef513", "Domain IIa" = "#05a8fa", "Domain IIb" = "#132cf5",
  "Domain III" = "#e705fa", "Fusion loop b" = "#05fa57", "Stem" ="#f4b4f3",
  "Transmembrane" = "#f33229"
)

# Ordering the domains for the legend
correct_domain_order_E <- E_domains_df$domain_name_E
E_domains_ordered_df <- E_domains_df %>%
  mutate(domain_name_E = factor(domain_name_E, levels = correct_domain_order_E))

#All E mutations lollipop plot
E_lollipop_plot <- ggplot(ALL_E, aes(x = position, y = percentage)) +
  
  # --- Draw the domain bar at the bottom ---
  # ymin and ymax are FIXED parameters, not aesthetics.
  geom_rect(
    data = E_domains_ordered_df,
    aes(xmin = start, xmax = end, fill = domain_name_E), # Only aesthetics that map to data columns 
    ymin = -10, ymax = 0,                               # Fixed parameters are outside aes()
    inherit.aes = FALSE,
    alpha = 0.8 # Increased alpha slightly for better visibility
  ) +
  
  
  geom_hline(yintercept = 0, color = "gray40") +
  geom_segment(aes(xend = position, yend = 0), linewidth = 0.8) +
  geom_point(aes(size = percentage), color = "black", fill = "white", shape = 21, stroke = 1.2) +


  # The label is now mapped to the domain_name_E column.
  geom_text(
    data = E_domains_ordered_df,
    aes(x = start + (end - start) / 2, y = -5, label = " "), # y is centered in the bar
    inherit.aes = FALSE,
    size = 2.5, angle = 90, # Rotated for better fit
    color = "black", check_overlap = TRUE
  ) +
  
  # Scales 
  scale_size_continuous(range = c(2, 12), name = "Occurrence (%)") +
  scale_fill_manual(values = domain_colors_E, name = "E Protein Domain") +
  scale_x_continuous(limits = c(-50, 550), name = "Amino Acid Position (1-495)", expand = c(0,0)) +
  scale_y_continuous(name = "Mutation Occurrence (%)", limits = c(-10, 105), expand = c(0, 0)) +
  
  # Labs 
  labs(
    title = "E Gene Mutation Hotspots in all DENV2 II_F1.1",
    subtitle = "Frequency and position of non-synonymous mutations"
  ) +
  
  # Theming and Legend Fix
  theme_classic(base_size = 14) +
  theme(
    panel.grid.major.y = element_line(color = "gray85", linetype = "dashed"),
    axis.line.x = element_line(),
    plot.title.position = "plot",
    legend.position = "right"
  ) +
  
  # --- legend transparency ---
  guides(fill = guide_legend(override.aes = list(alpha = 0.8)))


print(E_lollipop_plot)

# Save the plot
ggsave(
  "E_hotspots_IIF1.1.png",
  plot = E_lollipop_plot, width = 12, height = 7, dpi = 600
)


In [ ]:
# Lolipop plot of SL E mutations and occurence frequency
#--- Defining E Gene Domains and Colors (using your provided info) ---
E_domains_df_sl <- tibble(
  domain_name_E_sl = c("Domain Ia", "Domain Ib", "Domain Ic", "Fusion loop a", "Domain IIa", "Domain IIb", "Fusion loop b", "Domain III", "Stem", "Transmembrane"),
  start = c( 1, 131, 271, 70,  52, 192, 242, 299, 395, 450), # Corrected Vector Length
  end   = c(52, 192, 299, 82, 131, 271, 248, 394, 449, 495)  # Corrected Vector Length

)

domain_colors_E_sl <- c(
  "Domain Ia" = "#e4fa05", "Domain Ib" = "#fa8e06", "Domain Ic" = "#f4ee51",
  "Fusion loop a" ="#8ef513", "Domain IIa" = "#05a8fa", "Domain IIb" = "#132cf5",
  "Domain III" = "#e705fa", "Fusion loop b" = "#05fa57", "Stem" ="#f4b4f3",
  "Transmembrane" = "#f33229"
)

# Ordering the domains for the legend
correct_domain_order_Esl <- E_domains_df_sl$domain_name_E_sl
E_domains_ordered_dfsl <- E_domains_df_sl %>%
  mutate(domain_name_E_sl = factor(domain_name_E_sl, levels = correct_domain_order_Esl))

#All E mutations lollipop plot
Esl_lollipop_plot <- ggplot(SL_E, aes(x = position, y = percentage)) +
  
  # --- Draw the domain bar at the bottom ---
  # ymin and ymax are FIXED parameters, not aesthetics.
  geom_rect(
    data = E_domains_ordered_dfsl,
    aes(xmin = start, xmax = end, fill = domain_name_E_sl), # Only aesthetics that map to data columns 
    ymin = -10, ymax = 0,                               # Fixed parameters are outside aes()
    inherit.aes = FALSE,
    alpha = 0.8 # Increased alpha slightly for better visibility
  ) +
  
  
  geom_hline(yintercept = 0, color = "gray40") +
  geom_segment(aes(xend = position, yend = 0), linewidth = 0.8) +
  geom_point(aes(size = percentage), color = "black", fill = "white", shape = 21, stroke = 1.2) +


  # The label is now mapped to the domain_name_E column.
  geom_text(
    data = E_domains_ordered_dfsl,
    aes(x = start + (end - start) / 2, y = -5, label = " "), # y is centered in the bar
    inherit.aes = FALSE,
    size = 2.5, angle = 90, # Rotated for better fit
    color = "black", check_overlap = TRUE
  ) +
  
  # Scales 
  scale_size_continuous(range = c(2, 12), name = "Occurrence (%)") +
  scale_fill_manual(values = domain_colors_E_sl, name = "E Protein Domain") +
  scale_x_continuous(limits = c(-50, 550), name = "Amino Acid Position (1-495)", expand = c(0,0)) +
  scale_y_continuous(name = "Mutation Occurrence (%)", limits = c(-10, 105), expand = c(0, 0)) +
  
  # Labs 
  labs(
    title = "E Gene Mutation Hotspots in all Sri Lankan DENV2",
    subtitle = "Frequency and position of non-synonymous mutations"
  ) +
  
  # Theming and Legend Fix
  theme_classic(base_size = 14) +
  theme(
    panel.grid.major.y = element_line(color = "gray85", linetype = "dashed"),
    axis.line.x = element_line(),
    plot.title.position = "plot",
    legend.position = "right"
  ) +
  
  # --- legend transparency ---
  guides(fill = guide_legend(override.aes = list(alpha = 0.8)))


print(Esl_lollipop_plot)
# Save the plot
ggsave(
  "E_hotspots_SL.png",
  plot = Esl_lollipop_plot, width = 12, height = 7, dpi = 600
)

In [ ]:
#lollipop plot of all II_F1.1 NS1 mutation occurence and frequency
# --- Defining NS1 Domains with specific Y-coordinates for the bar ---
ns1_domains_df <- tibble(
  domain_name_1 = c("Beta-roll","Wing connector-1","Wing domain", "Wing connector-2", "Beta-ladder","Spaghetti loop", "C-Terminus"),
  start = c( 1,30,30,152,181,219,278),
  end =   c(29,37,180,180,352,279,352 ),
  # --- Define the y-range for the bar ---
  ymin = -8,
  ymax = -1
)

domain_colors_1 <- c(
  "Beta-roll" = "#a6cee3", "Wing connector-1" = "#a56aed", "Wing domain" = "#de5ef2",
  "Wing connector-2" = "#36f2e7", "Beta-ladder"  = "#f32957", "Spaghetti loop" ="#f4e709",
  "C-Terminus" = "#0647f4"
)

correct_domain_order_1 <- ns1_domains_df$domain_name_1
ns1_domains_ordered_df_1 <- ns1_domains_df %>%
  mutate(domain_name_1 = factor(domain_name_1, levels = correct_domain_order_1))


# --- Step 4: Build the plot with the domain bar ---
ns1_plot_bar <- ggplot(ALL_NS1, aes(x = position, y = percentage)) +
  
  # --- CHANGE HERE: The rectangles are now drawn as a thin bar at the bottom ---
  geom_rect(
    data = ns1_domains_ordered_df_1,
    aes(xmin = start, xmax = end, ymin = ymin, ymax = ymax, fill = domain_name_1),
    inherit.aes = FALSE,
    alpha = 0.8 # A bit more opaque for a solid bar look
  ) +
  
  # Lollipops 
  geom_hline(yintercept = 0, color = "gray40") +
  geom_segment(aes(xend = position, yend = 0), linewidth = 0.8) +
  geom_point(aes(size = percentage), color = "black", fill = "white", shape = 21, stroke = 1.2) +
  
  # --- Place the labels inside the new bar ---
  geom_text(
    data = ns1_domains_ordered_df_1,
    aes(x = start + (end - start) / 2, y = -4.5, label = " "), # Center text vertically in the bar
    inherit.aes = FALSE,
    size = 2.5, angle = 0, hjust = 0.5, color = "black", check_overlap = TRUE
  ) +
  
  # --- Scales of the lollipop heads ---
  scale_size_continuous(range = c(2, 10), name = "Occurrence (%)") +
  scale_fill_manual(values = domain_colors_1, name = "NS1 Protein Domain") +
  scale_x_continuous(limits = c(1, 352), name = "Amino Acid Position (1-352)", expand = c(0,0)) +
  scale_y_continuous(name = "Mutation Occurrence (%)", limits = c(-10, 105), expand = c(0, 0)) +
  
  # --- Theming and Legend Fix ---
  theme_classic(base_size = 14) +
  theme(
    panel.grid.major.y = element_line(color = "gray85", linetype = "dashed"),
    axis.line.x = element_line(), # Add a solid x-axis line at y=0 for separation
    axis.text.x = element_text(size = 12),
    plot.title.position = "plot",
    legend.position = "right"
  ) +
  # override.aes to match the legend to the bar's transparency
  guides(fill = guide_legend(override.aes = list(alpha = 0.8), ncol = 1)) +
  labs(
    title = "NS1 Mutation Hotspots of the DENV2 II_F1.1",
    subtitle = "Frequency and position of non-synonymous mutations"
  )

# Display the final plot
print(ns1_plot_bar)
# Save the plot
ggsave(
  "NS1_hotspots_IIF1.1.png",
  plot = ns1_plot_bar, width = 12, height = 7, dpi = 600
)

In [ ]:
#lollipop plot of Sri Lankan DENV2  NS1 mutation occurence and frequency
# --- Defining NS1 Domains with specific Y-coordinates for the bar ---
ns1sl_domains_df <- tibble(
  domain_name_1 = c("Beta-roll","Wing connector-1","Wing domain", "Wing connector-2", "Beta-ladder","Spaghetti loop", "C-Terminus"),
  start = c( 1,30,30,152,181,219,278),
  end =   c(29,37,180,180,352,279,352 ),
  # --- Define the y-range for the bar ---
  ymin = -8,
  ymax = -1
)

domain_colors_1 <- c(
  "Beta-roll" = "#a6cee3", "Wing connector-1" = "#a56aed", "Wing domain" = "#de5ef2",
  "Wing connector-2" = "#36f2e7", "Beta-ladder"  = "#f32957", "Spaghetti loop" ="#f4e709",
  "C-Terminus" = "#0647f4"
)

correct_domain_order_1 <- ns1sl_domains_df$domain_name_1
ns1_domains_ordered_df_1 <- ns1sl_domains_df %>%
  mutate(domain_name_1 = factor(domain_name_1, levels = correct_domain_order_1))


# --- Step 4: Build the plot with the domain bar ---
ns1sl_plot_bar <- ggplot(SL_NS1, aes(x = position, y = percentage)) +
  
  # --- CHANGE HERE: The rectangles are now drawn as a thin bar at the bottom ---
  geom_rect(
    data = ns1_domains_ordered_df_1,
    aes(xmin = start, xmax = end, ymin = ymin, ymax = ymax, fill = domain_name_1),
    inherit.aes = FALSE,
    alpha = 0.8 # A bit more opaque for a solid bar look
  ) +
  
  # Lollipops 
  geom_hline(yintercept = 0, color = "gray40") +
  geom_segment(aes(xend = position, yend = 0), linewidth = 0.8) +
  geom_point(aes(size = percentage), color = "black", fill = "white", shape = 21, stroke = 1.2) +
  
  # --- Place the labels inside the new bar ---
  geom_text(
    data = ns1_domains_ordered_df_1,
    aes(x = start + (end - start) / 2, y = -4.5, label = " "), # Center text vertically in the bar
    inherit.aes = FALSE,
    size = 2.5, angle = 0, hjust = 0.5, color = "black", check_overlap = TRUE
  ) +
  
  # --- Scales of the lollipop heads ---
  scale_size_continuous(range = c(2, 10), name = "Occurrence (%)") +
  scale_fill_manual(values = domain_colors_1, name = "NS1 Protein Domain") +
  scale_x_continuous(limits = c(1, 352), name = "Amino Acid Position (1-352)", expand = c(0,0)) +
  scale_y_continuous(name = "Mutation Occurrence (%)", limits = c(-10, 105), expand = c(0, 0)) +
  
  # --- Theming and Legend Fix ---
  theme_classic(base_size = 14) +
  theme(
    panel.grid.major.y = element_line(color = "gray85", linetype = "dashed"),
    axis.line.x = element_line(), # Add a solid x-axis line at y=0 for separation
    axis.text.x = element_text(size = 12),
    plot.title.position = "plot",
    legend.position = "right"
  ) +
  # override.aes to match the legend to the bar's transparency
  guides(fill = guide_legend(override.aes = list(alpha = 0.8), ncol = 1)) +
  labs(
    title = "NS1 Mutation Hotspots of Sri Lankan DENV2 ",
    subtitle = "Frequency and position of non-synonymous mutations"
  )

# Display the final plot
print(ns1sl_plot_bar)

# Save the plot
ggsave(
  "NS1_hotspots_SL.png",
  plot = ns1sl_plot_bar, width = 12, height = 7, dpi = 600
)

In [ ]:
#Comparing ALL_E and SL_E mutations
# Function to parse the mutation string into components
parse_mutation_components <- function(df) {
  matches <- str_match(df$mutation, "^([A-Z]):([A-Z])([0-9]+)([A-Z])$")
  df %>%
    mutate(
      domain_parsed = matches[, 2],       # e.g., "E"
      original_aa = matches[, 3],       # e.g., "I"
      position = as.numeric(matches[, 4]), # e.g., 4
      mutated_aa = matches[, 5]         # e.g., "V"
    )
}

# Apply the parsing function to both data frames
ALL_E_parsed <- parse_mutation_components(ALL_E)
SL_E_parsed <- parse_mutation_components(SL_E)

In [ ]:
#Comparing ALL_NS1 and SL_NS1 mutations
# Function to parse the mutation string into components
parse_mutation_components <- function(df) {
  matches <- str_match(df$mutation, "^(NS1):([A-Z])([0-9]+)([A-Z])$")
  df %>%
    mutate(
      domain_parsed = matches[, 2],       # e.g., "E"
      original_aa = matches[, 3],       # e.g., "I"
      position = as.numeric(matches[, 4]), # e.g., 4
      mutated_aa = matches[, 5]         # e.g., "V"
    )
}

# Apply the parsing function to both data frames
ALL_NS1_parsed <- parse_mutation_components(ALL_NS1)
SL_NS1_parsed <- parse_mutation_components(SL_NS1)

In [ ]:
# Find E mutations that are common to both datasets
common_mutations <- inner_join(
  ALL_E_parsed %>% select(mutation, domain, position, original_aa, mutated_aa, domain_parsed), # Select relevant columns to ensure accurate join
  SL_E_parsed %>% select(mutation, domain, position, original_aa, mutated_aa, domain_parsed),
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed") # Join by all parsed components for robustness
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed) %>% # Select final output columns
  distinct() # Ensure no accidental duplicates from join logic

# save common E mutations
print("Mutations in Common:")
print(common_mutations)
write.csv(common_mutations,"Common_mutations_E.csv")


In [ ]:
# Find NS1 mutations that are common to both datasets
common_mutations_NS1 <- inner_join(
  ALL_NS1_parsed %>% select(mutation, domain, position, original_aa, mutated_aa, domain_parsed), # Select relevant columns to ensure accurate join
  SL_NS1_parsed %>% select(mutation, domain, position, original_aa, mutated_aa, domain_parsed),
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed") # Join by all parsed components for robustness
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed) %>% # Select final output columns
  distinct() # Ensure no accidental duplicates from join logic

# save common NS1 mutations
print("NS1 Mutations in Common:")
print(common_mutations_NS1)
write.csv(common_mutations_NS1,"Common_mutations_NS1.csv")

In [ ]:
# Find mutations present in ALL_E but NOT in SL_E
diff_ALL_E_not_SL_E <- anti_join(
  ALL_E_parsed,
  SL_E_parsed,
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed")
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed)

# Find mutations present in SL_E but NOT in ALL_E
diff_SL_E_not_ALL_E <- anti_join(
  SL_E_parsed,
  ALL_E_parsed,
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed")
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed)

# Display different mutations
print("Mutations present only in ALL_E:")
print(diff_ALL_E_not_SL_E)

print("Mutations present only in SL_E:")
print(diff_SL_E_not_ALL_E)
write.csv(diff_SL_E_not_ALL_E,"mutations_unique_SLE.csv")

In [ ]:
# Find mutations present in ALL_NS1 but NOT in SL_NS1
diff_ALL_NS1_not_SL_NS1 <- anti_join(
  ALL_NS1_parsed,
  SL_NS1_parsed,
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed")
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed)

# Find mutations present in SL_NS1 but NOT in ALL_NS1
diff_SL_NS1_not_ALL_NS1 <- anti_join(
  SL_NS1_parsed,
  ALL_NS1_parsed,
  by = c("mutation", "domain", "position", "original_aa", "mutated_aa", "domain_parsed")
) %>%
  select(mutation, domain, position, original_aa, mutated_aa, domain_parsed)

# Display different mutations
print("Mutations present only in ALL_NS1:")
print(diff_ALL_NS1_not_SL_NS1)

print("Mutations present only in SL_NS1:")
print(diff_SL_NS1_not_ALL_NS1)
write.csv(diff_SL_NS1_not_ALL_NS1,"mutations_unique_SLNS1.csv")

In [ ]:
# Calculate the count of mutations per domain in diff_SL_E_not_ALL_E
domain_counts <- diff_SL_E_not_ALL_E %>%
  group_by(domain) %>%
  summarise(mutation_count = n()) %>%
  ungroup() # Ungroup for cleaner output

# Display the counts
print("Count of mutations per domain in diff_SL_E_not_ALL_E:")
print(domain_counts)
domain_counts_E <-diff_ALL_E_not_SL_E %>%
 group_by(domain)%>%
 summarise(mutation_count =n()) %>%
 ungroup()
 print(domain_counts_E)

In [ ]:
# Calculate the count of mutations per domain in diff_SL_E_not_ALL_E
domain_counts_NS1 <- diff_SL_NS1_not_ALL_NS1 %>%
  group_by(domain) %>%
  summarise(mutation_count = n()) %>%
  ungroup() # Ungroup for cleaner output

# Display the counts
print("Count of mutations unique to SL NS1 per domain :")
print(domain_counts_NS1)

In [ ]:
#No of mutations per domain II_F1.1 E
# Make sure the tidyverse is loaded for the dplyr functions
library(tidyverse)

# --- Define the Total Number of Samples (The Denominator) ---
total_samples <- 656


# --- Analysis Pipeline ---
domain_prevalence_summary <- ALL_E %>%
  
  # Step 1: Group by domain to aggregate all mutations within it.
  group_by(domain) %>%
  
  # Step 2: Calculate the numerator for each domain.
  # This is the total number of times mutations in this domain were observed.
  # We also count the number of unique mutation types for context.
  summarise(
    number_of_unique_mutations = n(),
    total_observations_in_domain = sum(n_samples_with_mutation),
    .groups = "drop" # Ungroup for the next step
  ) %>%
  
  # Step 3: Calculate the final prevalence percentage for each domain.
  mutate(
    # The numerator (sum of counts in the domain) is divided by the
    # fixed total number of samples in the entire study.
    prevalence_percentage = (total_observations_in_domain / total_samples) * 100,
    
    # (Optional) Create a nicely formatted string for display
    percentage_display = sprintf("%.2f%%", prevalence_percentage)
  ) %>%
  
  # Step 4: Sort the results to see the most prevalent domains first.
  arrange(desc(prevalence_percentage))

# --- Print the Final Summary Table ---
print("--- Summary of Mutation Prevalence Per Domain ---")
print(domain_prevalence_summary)





In [ ]:
#mutations per domain in Sri Lankan samples E
# --- 1. Calculate the Denominator: The Sum of All Mutation Frequencies ---
# This represents the total mutation frequency across all unique mutations.
total_mutation_frequency_SLE <- sum(SL_E$percentage)

print(paste("Total sum of all mutation frequencies (%):", total_mutation_frequency_SLE))


# --- 2. Calculate Per-Domain Contribution to Total Frequency ---

domain_frequency_summary_SLE <- SL_E %>%
  
  # Group by the domain to aggregate all mutations within each one
  group_by(domain) %>%
  
  # Calculate the sum of percentages for each domain (the numerator)
  # and also count the number of unique mutations for reference.
  summarise(
    number_of_unique_mutations = n(),
    sum_of_percentages = sum(percentage),
    .groups = "drop" # Drop grouping for the next step
  ) %>%
  
  # Now, calculate the percentage contribution of each domain
  mutate(
    percentage_of_total_frequency = (sum_of_percentages / total_mutation_frequency_SLE) * 100,
    
    # (Optional) Create a nicely formatted string for display
    percentage_display = sprintf("%.2f%%", percentage_of_total_frequency)
  ) %>%
  
  # Sort the final table by the contribution percentage
  arrange(desc(percentage_of_total_frequency))

# --- Print the Final Summary Table ---
print("--- Summary of Domain Contribution to Total Mutation Frequency ---")
print(domain_frequency_summary_SLE)


In [ ]:
#mutations per domain in Sri Lankan samples NS1
# --- 1. Calculate the Denominator: The Sum of All Mutation Frequencies ---
# This represents the total mutation frequency across all unique mutations.
total_mutation_frequency_SLNS1 <- sum(SL_NS1$percentage)

print(paste("Total sum of all mutation frequencies (%):", total_mutation_frequency_SLNS1))


# --- 2. Calculate Per-Domain Contribution to Total Frequency ---

domain_frequency_summary_SLNS1 <- SL_NS1 %>%
  
  # Group by the domain to aggregate all mutations within each one
  group_by(domain) %>%
  
  # Calculate the sum of percentages for each domain (the numerator)
  # and also count the number of unique mutations for reference.
  summarise(
    number_of_unique_mutations = n(),
    sum_of_percentages = sum(percentage),
    .groups = "drop" # Drop grouping for the next step
  ) %>%
  
  # Now, calculate the percentage contribution of each domain
  mutate(
    percentage_of_total_frequency = (sum_of_percentages / total_mutation_frequency_SLNS1) * 100,
    
    # (Optional) Create a nicely formatted string for display
    percentage_display = sprintf("%.2f%%", percentage_of_total_frequency)
  ) %>%
  
  # Sort the final table by the contribution percentage
  arrange(desc(percentage_of_total_frequency))

# --- Print the Final Summary Table ---
print("--- Summary of Domain Contribution to Total Mutation Frequency ---")
print(domain_frequency_summary_SLNS1)


In [ ]:
SL_NS1_mutation_matrix <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/SL_NS1_mutations_matrix.csv')
SL_E_mutation_matrix <- read.csv ('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/SL_E_mutations_matrix.csv')

In [ ]:
head(SL_NS1_mutation_matrix)


In [ ]:
#get mutation counts E SL
total_mutation_count <- SL_E_mutation_matrix %>%
  
  # Step 1: Isolate the numeric mutation columns by removing the 'seqName' column.
  # The minus sign (-) means "exclude this column".
  select(-seqName) %>%
  
  # Step 2: Calculate the grand total by summing every element in the remaining data.
  sum()

# --- Print the Final Result ---
print(paste("The total count of all mutations in all samples is:", total_mutation_count))

In [ ]:
#get mutation counts NS1 SL
total_mutation_count <- SL_NS1_mutation_matrix %>%
  
  # Step 1: Isolate the numeric mutation columns by removing the 'seqName' column.
  # The minus sign (-) means "exclude this column".
  select(-seqName) %>%
  
  # Step 2: Calculate the grand total by summing every element in the remaining data.
  sum()

# --- Print the Final Result ---
print(paste("The total count of all mutations in all samples is:", total_mutation_count))

In [ ]:
All_mutations_matrix <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/All_mutations.csv')
head(All_mutations_matrix)

In [ ]:
# --- 1. Create the E Gene Matrix (ALL_E_matrix) ---

ALL_E_matrix <- All_mutations_matrix %>%
  
  # Select the 'seqName' column AND all columns that start with "E."
  select(seqName, starts_with("E."))

# --- 2. Create the NS1 Gene Matrix (ALL_NS1_matrix) ---

ALL_NS1_matrix <- All_mutations_matrix %>%
  
  # Select the 'seqName' column AND all columns that start with "NS1."
  select(seqName, starts_with("NS1."))


# --- 3. Save the New Matrices to .csv Files ---

# Use readr::write_csv() to save the files to your current working directory.
write_csv(ALL_E_matrix, "ALL_E_matrix.csv")
write_csv(ALL_NS1_matrix, "ALL_NS1_matrix.csv")

In [ ]:
#get mutation counts E II_F1.1
total_mutation_count <- ALL_E_matrix %>%
  
  # Step 1: Isolate the numeric mutation columns by removing the 'seqName' column.
  # The minus sign (-) means "exclude this column".
  select(-seqName) %>%
  
  # Step 2: Calculate the grand total by summing every element in the remaining data.
  sum()

# --- Print the Final Result ---
print(paste("The total count of all mutations in all samples is:", total_mutation_count))

In [ ]:
#get mutation counts NS1 II_F1.1
total_mutation_count <- ALL_NS1_matrix %>%
  
  # Step 1: Isolate the numeric mutation columns by removing the 'seqName' column.
  # The minus sign (-) means "exclude this column".
  select(-seqName) %>%
  
  # Step 2: Calculate the grand total by summing every element in the remaining data.
  sum()

# --- Print the Final Result ---
print(paste("The total count of all mutations in all samples is:", total_mutation_count))

In [ ]:
all_mutations <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/All_mutations.csv')

In [ ]:
#filter Indian samples
df_IN_DENV2 <- all_mutations %>%
  filter(str_detect(seqName, "India"))

# Verify the new subset by checking its first few rows
# 
head(df_IN_DENV2)

# 
# to see how many Indian sequences were found
nrow(df_IN_DENV2)

In [ ]:
#Indian sequences E and NS1
E_India_mutations_matrix <- df_IN_DENV2 %>%
  select(seqName, contains("E."))
NS1_India_mutations_matrix <- df_IN_DENV2 %>%
  select(seqName, contains("NS1."))
  head(E_India_mutations_matrix)
  head(NS1_India_mutations_matrix)

In [ ]:
# Define the total number of samples (sequences)
total_samples <- nrow(E_India_mutations_matrix)

# Calculate mutation frequencies
E_India_freq_table <- E_India_mutations_matrix %>%
  # 1. Select only the mutation columns
  select(-seqName) %>%
  
  # 2. Calculate the sum (count) for each mutation column
  summarise(across(everything(), sum)) %>%
  
  # 3. Pivot the data from wide to long format
  # This creates one row for each mutation
  pivot_longer(
    cols = everything(),
    names_to = "Mutation",
    values_to = "Count"
  ) %>%
  
  # 4. Calculate the frequency percentage
  mutate(Frequency_Percent = (Count / total_samples) * 100) %>%
  
  # 5. Sort the table by frequency in descending order
  arrange(desc(Frequency_Percent))

# Print the resulting table
print("--- E Region Mutation Frequencies in Indian F1.1 ---")
print(E_India_freq_table)


In [ ]:
# Define the total number of samples (sequences)
total_samples_NS1 <- nrow(NS1_India_mutations_matrix)

# Calculate mutation frequencies
NS1_India_freq_table <- NS1_India_mutations_matrix %>%
  # 1. Select only the mutation columns
  select(-seqName) %>%
  
  # 2. Calculate the sum (count) for each mutation column
  summarise(across(everything(), sum)) %>%
  
  # 3. Pivot the data from wide to long format
  # This creates one row for each mutation
  pivot_longer(
    cols = everything(),
    names_to = "Mutation",
    values_to = "Count"
  ) %>%
  
  # 4. Calculate the frequency percentage
  mutate(Frequency_Percent = (Count / total_samples_NS1) * 100) %>%
  
  # 5. Sort the table by frequency in descending order
  arrange(desc(Frequency_Percent))

# Print the resulting table
print("--- NS1Region Mutation Frequencies in Indian F1.1 ---")
print(NS1_India_freq_table)


In [ ]:
E_domains_df <- tibble(
  domain_name_E = c("Domain Ia", "Domain Ib", "Domain Ic", "Fusion loop a", "Domain IIa", "Domain IIb", "Fusion loop b", "Domain III", "Stem", "Transmembrane"),
  start = c( 1, 131, 271, 70,  52, 192, 242, 299, 395, 450),
  end   = c(51, 191, 298, 82, 130, 270, 248, 394, 449, 495)
)

# --- Analysis Pipeline ---

total_samples <- nrow(E_India_mutations_matrix)

E_India_freq_table_with_domains <- E_India_mutations_matrix %>%
  select(-seqName) %>%
  summarise(across(everything(), sum)) %>%
  pivot_longer(
    cols = everything(),
    names_to = "Mutation",
    values_to = "Count"
  ) %>%
  
  # --- CORRECTED STEP ---
  # 1. Use str_extract to find the number (as text). It will return NA if no number is found.
  # 2. Use as.numeric to convert the text to a number. NA remains NA.
  # This avoids the parsing failure warnings.
  mutate(Position = as.numeric(str_extract(Mutation, "\\d+"))) %>%
  
  # The rest of the pipeline remains the same
  left_join(E_domains_df, by = join_by(between(Position, start, end))) %>%
  mutate(Frequency_Percent = (Count / total_samples) * 100) %>%
  select(
    Mutation,
    Position,
    Domain = domain_name_E,
    Count,
    Frequency_Percent
  ) %>%
  arrange(desc(Frequency_Percent))

# Print the resulting table
print("--- E Region Mutation Frequencies in Indian F1.1 (with Domains) ---")
print(E_India_freq_table_with_domains)

In [ ]:

# 2. Your NS1 domain definition data frame
ns1_domains_df <- tibble(
  domain_name_1 = c("Beta-roll", "Wing connector-1", "Wing domain", "Wing connector-2", "Beta-ladder", "Spaghetti loop", "C-Terminus"),
  start = c(1, 30, 38, 152, 181, 219, 278), # Adjusted Wing domain start to be non-overlapping
  end   = c(29, 37, 180, 180, 352, 277, 352), # Adjusted Spaghetti loop end for clarity
  ymin = -8,
  ymax = -1
)


total_samples_ns1 <- nrow(NS1_India_mutations_matrix)

NS1_India_freq_table_with_domains <- NS1_India_mutations_matrix %>%
  select(-seqName) %>%
  summarise(across(everything(), sum)) %>%
  pivot_longer(
    cols = everything(),
    names_to = "Mutation",
    values_to = "Count"
  ) %>%
  
 
  mutate(Position = as.numeric(str_match(Mutation, "NS1\\.[A-Z](\\d+)[A-Z]")[, 2])) %>%
  
  # The rest of the pipeline remains the same
  left_join(ns1_domains_df, by = join_by(between(Position, start, end))) %>%
  mutate(Frequency_Percent = (Count / total_samples_ns1) * 100) %>%
  select(
    Mutation,
    Position,
    Domain = domain_name_1,
    Count,
    Frequency_Percent
  ) %>%
  arrange(desc(Frequency_Percent))

# Print the resulting table
print("--- NS1 Region Mutation Frequencies in Indian F1.1 (with Domains) ---")
print(NS1_India_freq_table_with_domains)

In [ ]:
write.csv(E_India_freq_table_with_domains,"India_E_mutations.csv")
write.csv(NS1_India_freq_table_with_domains,"India_NS1_mutations.csv")


In [ ]:
df_E <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/E_ALL_SL_IN.csv')
df_NS1 <- read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/NS1_ALL_SL_IN.csv')


In [ ]:
#Comparison of E mutations in all F1.1, Sri Lankan and Indian sequences
library(tidyverse)
# extract these columns into clean vectors, removing any NAs
all_mutations <- na.omit(df_E$X...mutation_all)
sl_mutations  <- na.omit(df_E$mutation_SL)
india_mutations <- na.omit(df_E$mutation_India)

# 1. Get a complete list of all unique mutations from all groups
all_unique_mutations <- union(union(all_mutations, sl_mutations), india_mutations)

# 2. Create a summary tibble
comparison_df <- tibble(mutation = all_unique_mutations) %>%
  mutate(
    in_All = as.integer(mutation %in% all_mutations),
    in_SL = as.integer(mutation %in% sl_mutations),
    in_India = as.integer(mutation %in% india_mutations)
  ) %>%
  # Optional: Add a column that sums the occurrences
  mutate(total_groups = in_All + in_SL + in_India) %>%
  # Optional: Arrange to see the most shared mutations first
  arrange(desc(total_groups))

print("Comprehensive Summary Table:")
print(comparison_df)
write.csv(comparison_df,"E_mutations_comparison.csv")

In [ ]:
#Comparison of NS1 mutations in all F1.1, Sri Lankan and Indian sequences
library(tidyverse)
# extract these columns into clean vectors, removing any NAs
all_mutations_NS1 <- na.omit(df_NS1$X...mutation_all)
sl_mutations_NS1  <- na.omit(df_NS1$mutation_SL)
india_mutations_NS1 <- na.omit(df_NS1$mutation_India)

# 1. Get a complete list of all unique mutations from all groups
all_unique_mutations_NS1 <- union(union(all_mutations_NS1, sl_mutations_NS1), india_mutations_NS1)

# 2. Create a summary tibble
comparison_df_NS1 <- tibble(mutation = all_unique_mutations_NS1) %>%
  mutate(
    in_All = as.integer(mutation %in% all_mutations_NS1),
    in_SL = as.integer(mutation %in% sl_mutations_NS1),
    in_India = as.integer(mutation %in% india_mutations_NS1)
  ) %>%
  # Optional: Add a column that sums the occurrences
  mutate(total_groups = in_All + in_SL + in_India) %>%
  # Optional: Arrange to see the most shared mutations first
  arrange(desc(total_groups))

print("Comprehensive Summary Table:")
print(comparison_df_NS1)
write.csv(comparison_df_NS1,"NS1_mutations_comparison.csv")

In [ ]:
#E mutations comparison
#common to SL and India 
print("common to SL and India")
filter(comparison_df, in_SL == 1 , in_India == 1)
#unique to India and Sri Lanka
print("unique to only India and Sri Lanka")
filter(comparison_df, in_SL==1,in_India==1, in_All==0)
#unique to SL compared to India
print("unique to SL compared to India")
filter(comparison_df, in_SL==1, in_India==0)
#totally unique to SL
print("totally unique to SL")
filter(comparison_df, in_SL==1, in_India==0,in_All==0)

In [ ]:
#NS1 mutations comparison
#common to all
print("common to all")
filter(comparison_df_NS1, in_SL == 1 , in_India == 1, in_All==1)
#unique to India and Sri Lanka
print("unique to only India and Sri Lanka")
filter(comparison_df_NS1, in_SL==1,in_India==1, in_All==0)
#unique to SL compared to India
print("unique to SL compared to India")
filter(comparison_df_NS1, in_SL==1, in_India==0)
#totally unique to SL
print("totally unique to SL")
filter(comparison_df_NS1, in_SL==1, in_India==0,in_All==0)
#Totally unique to India
print("totally unique to India")
filter(comparison_df_NS1, in_SL==0,in_India==1,in_All==0)

In [ ]:
e_domains_df <- tibble(
  domain_name = c("Domain Ia","Domain IIa", "Domain Ib", "Domain IIb", "Domain Ic","Domain III", "Stem","Transmembrane"),
  start       = c(1,  53, 132,193,272,300,395,450),
  end         = c(52,131,192,271,299,394,449,495)
)

# --- Step 1: Prepare Data for Heatmap Tiles (Your code here was correct) ---

heatmap_tiles <- comparison_df %>%
  select(mutation, `SL Clade` = in_SL,`India clade`=in_India, `F1.1 Clade` = in_All) %>%
  mutate(Position = as.numeric(str_match(mutation, "[A-Z]:[A-Z](\\d+)[A-Z]")[, 2])) %>%
  filter(!is.na(Position)) %>%
  pivot_longer(
    cols = c(`SL Clade`,`India clade`, `F1.1 Clade`),
    names_to = "Clade",
    values_to = "Status"
  ) %>%
  filter(Status == 1)

# --- Step 2: Prepare Data for Domain Annotation Layer ---
# This code now works because e_domains_df is a proper data frame.
domain_annotations <- e_domains_df %>%
  mutate(
    midpoint = start + (end - start) / 2,
    fill_factor = factor(row_number() %% 2)
  )

# --- Step 3: Build the Advanced Plot ---

domain_y_min <- 0.3
domain_y_max <- 0.7

ggplot() +
  geom_rect(
    data = domain_annotations,
    aes(xmin = start, xmax = end, ymin = domain_y_min, ymax = domain_y_max, fill = fill_factor),
    alpha = 0.5
  ) +
  # --- FIX 2: Correct column name for the label ---
  # The aes call for geom_text now correctly finds the 'domain_name' column.
  geom_text(
    data = domain_annotations,
    aes(x = midpoint, y = (domain_y_min + domain_y_max) / 2, label = domain_name),
    size = 2,
    color = "black"
  ) +
  geom_tile(
    data = heatmap_tiles,
    aes(x = Position, y = Clade),
    fill = "darkblue",
    height = 0.5,
    width = 1.5
  ) +
  scale_fill_manual(values = c("gray80", "gray65"), guide = "none") +
  # --- FIX 3: Correctly access the 'end' column ---
  # This now works because e_domains_df is a data frame with an 'end' column.
  scale_x_continuous(
    breaks = seq(0, max(e_domains_df$end, na.rm = TRUE) + 10, by = 10),
    expand = c(0.01, 0.01)
  ) +
  scale_y_discrete(expand = expansion(add = c(0.8, 0.2))) +
  theme_minimal() +
  labs(
    title = "Positional Comparison of Mutations in E Protein",
    subtitle = "Comparing SL,Indian  and F1.1 Clades",
    x = "Amino Acid Position",
    y = ""
  ) +
  theme(
    panel.grid.major.y = element_blank(),
    panel.grid.minor.x = element_blank(),
    axis.text.x = element_text(angle = 90, vjust = 0.5, size = 8)
  )


In [ ]:
comparison_df_NS1_clean <-read.csv('/Users/dinukaariyaratne/Library/Mobile Documents/com~apple~CloudDocs/Desktop/PhD USJP/PhD_Thesis/DENV2 paper/NS1_mutations_comparison.csv')
#NS1 comparison heatmap
NS1_domains_df <- tibble(
  domain_name_NS1 = c("Beta-roll","Wing domain",  "Beta-ladder"),
  start = c( 1, 30,181),
  end =   c(29,180,352),
)

# --- Step 1: Prepare Data for Heatmap Tiles (Your code here was correct) ---

heatmap_tiles_NS1 <- comparison_df_NS1_clean %>%
  select(mutation, `SL Clade` = in_SL,`India clade`=in_India, `F1.1 Clade` = in_All) %>%
  mutate(Position = as.numeric(str_match(mutation, "NS1:[A-Z](\\d+)[A-Z]")[, 2])) %>%
  filter(!is.na(Position)) %>%
  pivot_longer(
    cols = c(`SL Clade`,`India clade`, `F1.1 Clade`),
    names_to = "Clade",
    values_to = "Status"
  ) %>%
  filter(Status == 1)

# --- Step 2: Prepare Data for Domain Annotation Layer ---
# This code now works because e_domains_df is a proper data frame.
domain_annotations <- NS1_domains_df %>%
  mutate(
    midpoint = start + (end - start) / 2,
    fill_factor = factor(row_number() %% 2)
  )

# --- Step 3: Build the Advanced Plot ---

domain_y_min <- 0.3
domain_y_max <- 0.7

ggplot() +
  geom_rect(
    data = domain_annotations,
    aes(xmin = start, xmax = end, ymin = domain_y_min, ymax = domain_y_max, fill = fill_factor),
    alpha = 0.5
  ) +
  # --- FIX 2: Correct column name for the label ---
  # The aes call for geom_text now correctly finds the 'domain_name' column.
  geom_text(
    data = domain_annotations,
    aes(x = midpoint, y = (domain_y_min + domain_y_max) / 2, label = domain_name_NS1),
    size = 2,
    color = "black"
  ) +
  geom_tile(
    data = heatmap_tiles_NS1,
    aes(x = Position, y = Clade),
    fill = "darkblue",
    height = 0.5,
    width = 1.5
  ) +
  scale_fill_manual(values = c("gray80", "gray65"), guide = "none") +
  # --- FIX 3: Correctly access the 'end' column ---
  # This now works because e_domains_df is a data frame with an 'end' column.
  scale_x_continuous(
    breaks = seq(0, max(NS1_domains_df$end, na.rm = TRUE) + 10, by = 10),
    expand = c(0.01, 0.01)
  ) +
  scale_y_discrete(expand = expansion(add = c(0.8, 0.2))) +
  theme_minimal() +
  labs(
    title = "Positional Comparison of Mutations in NS1 Protein",
    subtitle = "Comparing Sri Lanka, India and the total F1.1 Clade",
    x = "Amino Acid Position",
    y = ""
  ) +
  theme(
    panel.grid.major.y = element_blank(),
    panel.grid.minor.x = element_blank(),
    axis.text.x = element_text(angle = 90, vjust = 0.5, size = 8)
  )